# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,aasiaat,68.7098,-52.8699,-22.28,96,100,1.91,GL,1738716350
1,1,bryanka,48.5110,38.6722,-3.70,87,83,4.86,UA,1738716351
2,2,itaocara,-21.6692,-42.0761,23.71,91,100,1.78,BR,1738716353
3,3,grytviken,-54.2811,-36.5092,3.31,86,53,9.59,GS,1738716354
4,4,adamstown,-25.0660,-130.1015,25.20,77,5,5.50,PN,1738716355


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [45]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 500   
)
# Display the map

map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[(city_data_df["Max Temp"] < 27) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values

ideal_cities_df = ideal_cities_df.dropna()

# Display sample data

ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,i-n-salah,27.1935,2.4607,12.53,21,0,7.54,DZ,1738716364
28,28,margaret river,-33.9500,115.0667,23.01,48,0,1.86,AU,1738716384
29,29,hilo,19.7297,-155.0900,25.85,73,0,4.63,US,1738716385
41,41,hawaiian paradise park,19.5933,-154.9731,25.73,69,0,4.63,US,1738716402
47,47,saint-pierre,-21.3393,55.4781,26.82,83,0,3.09,RE,1738716409


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df["Hotel Name"] = ""

# Display sample data

hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,aasiaat,GL,68.7098,-52.8699,96,
1,bryanka,UA,48.5110,38.6722,87,
2,itaocara,BR,-21.6692,-42.0761,91,
3,grytviken,GS,-54.2811,-36.5092,86,
4,adamstown,PN,-25.0660,-130.1015,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
aasiaat - nearest hotel: SØMA
bryanka - nearest hotel: Nika Hotel
itaocara - nearest hotel: Hotel Itaocara
grytviken - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
la'ie - nearest hotel: No hotel found
waitangi - nearest hotel: Hotel Chathams
la passe - nearest hotel: L'Estuaire
merizo village - nearest hotel: No hotel found
bredasdorp - nearest hotel: Victoria Hotel
port lincoln - nearest hotel: Boston Hotel
i-n-salah - nearest hotel: باجودة
blackmans bay - nearest hotel: Villa Howden
ilebo - nearest hotel: Hôtel des Palmes
tomar - nearest hotel: Vila Galé Collection Tomar
taltal - nearest hotel: Hostería Taltal
utrik - nearest hotel: No hotel found
atafu village - nearest hotel: No hotel found
praia da vitoria - nearest hotel: Salles
papatowai - nearest hotel: No hotel found
afaahiti - nearest hotel: Omati Lodge
yellowknife - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
puerto deseado - near

,City,Country,Lat,Lng,Humidity,Hotel Name
0,aasiaat,GL,68.7098,-52.8699,96,SØMA
1,bryanka,UA,48.5110,38.6722,87,Nika Hotel
2,itaocara,BR,-21.6692,-42.0761,91,Hotel Itaocara
3,grytviken,GS,-54.2811,-36.5092,86,No hotel found
4,adamstown,PN,-25.0660,-130.1015,77,No hotel found
...,...,...,...,...,...,...
563,el calafate,AR,-50.3408,-72.2768,61,Hostería Las Avutardas
564,fort stockton,US,30.8940,-102.8793,41,La Quinta Inn
565,cambria,US,35.5641,-121.0808,94,Olallieberry Inn
566,wynyard,AU,-41.0000,145.7167,61,Wynyard Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [44]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 500,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map

map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)